**News Headlines Dataset For Sarcasm Detection**
  + Liên kết Kaggle với Colab
  + Đọc dữ liệu file json bằng thư viện Pandas
  + Tạo ra 1 bag words, chứa tất cả những từ suất hiện trong các câu băng thư viện tensorflow
  + Chuẩn hóa tất cả các câu trong bộ dữ liệu và chia bộ dữ liệu thành bộ train và test
  + Dùng Multinomial Naive Bayes model để train dữ liệu
  + Đánh giá model

In [2]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [4]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# Find "News Headlines Dataset For Sarcasm Detection" dataset
# https://github.com/Kaggle/kaggle-api
!kaggle datasets list -s "News Headlines Dataset For Sarcasm Detection"

ref                                                            title                                                size  lastUpdated          downloadCount  
-------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
rmisra/news-headlines-dataset-for-sarcasm-detection            News Headlines Dataset For Sarcasm Detection          3MB  2019-07-03 23:52:57          19541  
rmisra/news-category-dataset                                   News Category Dataset                                25MB  2018-12-02 04:09:45          16463  
rmisra/clothing-fit-dataset-for-size-recommendation            Clothing Fit Dataset for Size Recommendation         40MB  2018-08-21 19:00:16           3677  
rmisra/imdb-spoiler-dataset                                    IMDB Spoiler Dataset                                331MB  2019-05-22 04:44:43           1066  
anasqais/news-headlines-dataset-for-sarcasm-de

In [6]:
# Copy the stackoverflow data set locally.
!kaggle datasets download -d rmisra/"news-headlines-dataset-for-sarcasm-detection"                         

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [7]:
!unzip news-headlines-dataset-for-sarcasm-detection.zip

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


In [8]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
# Read data from json file
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)
X = data.iloc[:, 1]
y = data.iloc[:, 2]

print('shape of data: ',data.shape)

# Init words bag
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X)

# Len words bag
len_wordBag = len(tokenizer.word_index)
print("Vocabulary size", len_wordBag)

shape of data:  (26709, 3)
Vocabulary size 29656


In [27]:
# Normalize train input
def Normalize(train_input):
  arr = np.zeros(shape=(len_wordBag))
  for i in train_input:
    if len(i) != 0:
      arr[i[0]] += 1
  return arr

In [28]:
# Create data train
a = 200
b = 500
X_train = tokenizer.texts_to_sequences(X[a])
X_train = Normalize(X_train)
for i in range(a+ 1, b):
  X_train = np.vstack((X_train, Normalize(tokenizer.texts_to_sequences(X[i]))))

y_train = y[a:b]

In [29]:
# Create data test
c = 200
d = 500
X_test = tokenizer.texts_to_sequences(X[c])
X_test = Normalize(X_test)
for i in range(c+1, d):
  X_test = np.vstack((X_test, Normalize(tokenizer.texts_to_sequences(X[i]))))

y_test = y[c:d]

In [32]:
# Create multinomial Naive Bayes model 
model = MultinomialNB()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# How to good model
print('Score = ',model.score(X_test, y_test))
print('Confusion matrix: ')
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Score =  0.58
Confusion matrix: 
[[174   0]
 [126   0]]
              precision    recall  f1-score   support

           0       0.58      1.00      0.73       174
           1       0.00      0.00      0.00       126

    accuracy                           0.58       300
   macro avg       0.29      0.50      0.37       300
weighted avg       0.34      0.58      0.43       300



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
